In [1]:
%run ../data_tools.ipynb
%run ../create_metrics.ipynb

      T_ID   GDAY_DS  HEADER_NO   P_ID  START_CK  BAT_ORDER_NO   PA   AB  RBI  \
0       HH  20160401          0  60404         0             3    1    1    0   
1       HH  20160401          0  62700         1             9    2    2    0   
2       HH  20160401          0  64086         1             7    6    4    0   
3       HH  20160401          0  66740         1             5    6    6    0   
4       HH  20160401          0  71347         1             2    6    6    1   
...    ...       ...        ...    ...       ...           ...  ...  ...  ...   
18679   WO  20161009          0  74215        91           374  402  341   80   
18680   WO  20161009          0  78168       139           177  646  560   63   
18681   WO  20161009          0  79130        15           251   80   66    7   
18682   WO  20161009          0  79300        13           429  106   91    9   
18683   WO  20161009          0  79365       122           965  454  411   70   

       RUN  ...  BB  IB  HP

In [2]:
import requests
import pandas as pd

from bs4 import BeautifulSoup
player_list = load_data(2020)["player"]
team_list = load_data(2020)["team"]

In [3]:
req = requests.get("http://old.statiz.co.kr/player.php?opt=1")
html = req.text
soup = BeautifulSoup(html, "html.parser")
player_url = soup.select(".table_stz a")

default_url = "http://old.statiz.co.kr/player.php?opt=3&"
default_pa_url = "http://old.statiz.co.kr/player.php?opt=6&"
default_money_url = "http://old.statiz.co.kr/player.php?opt=11&"
batter = pd.DataFrame()
pitcher = pd.DataFrame()
batter_pa = pd.DataFrame()

for i in range(len(player_url)) :
    req = requests.get(default_url + player_url[i]["href"].split("?")[1])
    html = req.text
    soup = BeautifulSoup(html, "html.parser")
    table = soup.select("table", {"class" : "table_stz"})
    res = pd.read_html(str(table))
    result = res[0].loc[1:].reset_index(drop = True) 
    team_info = soup.findAll("div", {"id" : ["minimenu", "minimenu_list"]})
    
    req = requests.get(default_money_url + player_url[i]["href"].split("?")[1])
    html = req.text
    soup = BeautifulSoup(html, "html.parser")
    table = soup.select("table", {"class" : "table_stz"})
    money = pd.read_html(str(table))

    
    if result.loc[0][3] == "타순" :
        result = result[result.columns[:23]].reset_index(drop = True)
        result["NAME"] = player_url[i].text
        result["AGE"] = team_info[1].text
        result["T_ID"] = team_info[13].text
        result["MONEY"] = money[0].iloc[-1, 1]
        batter = pd.concat([batter, result], axis = 0, ignore_index = True)
        batter_columns = result.columns
        
        req = requests.get(default_pa_url + player_url[i]["href"].split("?")[1] + "&re=0&da=1&year=2020&plist=&pdate=")
        html = req.text
        soup = BeautifulSoup(html, "html.parser")
        table = soup.select("table", {"class" : "table_stz"})
        res = pd.read_html(str(table))
        pa_res = res[0][res[0].columns[:5]][1:]
        pa_res.columns = pa_res.loc[1]
        pa_res = pa_res[1:].reset_index(drop = True)
        batter_pa = pd.concat([batter_pa, pa_res])
        
    elif result.loc[0][3] ==  "선발":
        result = result[result.columns[:19]].reset_index(drop = True)
        result["NAME"] = player_url[i].text
        result["AGE"] = team_info[1].text
        result["T_ID"] = team_info[13].text
        result["MONEY"] = money[0].iloc[-1, 1]
        pitcher = pd.concat([pitcher, result], axis = 0, ignore_index = True)

In [4]:
batter.columns = ["GDAY_DS", "VS_T_ID", "RES", "BAT_ORDER_NO", "POSITION", "START_CK", "AB", "RUN",
                "HIT", "H2", "H3", "HR", "TB", "RBI", "SB", "CS", "BB", "HP", "IB", "KK", "GD", "SH", "SF", "NAME", "AGE", "T_ID", "MONEY"]
batter = batter[batter["VS_T_ID"] != "상대"]
pitcher.columns = ["GDAY_DS", "VS_T_ID", "RES", "START_CK", "INN2",
                   "R", "ER", "PA", "AB", "HIT", "H2", "H3", "HR",
                   "BB", "IB", "HP", "KK", "BF", "WHIP", "NAME", "AGE", "T_ID", "MONEY"]
pitcher = pitcher[pitcher["VS_T_ID"] != "상대"]

In [5]:
pitcher.loc[:, "GDAY_DS"] = pitcher["GDAY_DS"].replace("\-", '', regex = True).astype(int)
batter.loc[:, "GDAY_DS"] = batter["GDAY_DS"].replace("\-", '', regex = True).astype(int)
pitcher = pitcher[pitcher["GDAY_DS"] > 719].reset_index(drop = True)
batter = batter[batter["GDAY_DS"] > 719].reset_index(drop = True)

In [6]:
batter["GDAY_DS"] = "20200" + batter["GDAY_DS"].astype(str)
for i in range(len(team_list)) :
    batter.loc[team_list["T_NM"][i] == batter["T_ID"], ["T_ID"]] = team_list["T_ID"][i]

In [7]:
pitcher["GDAY_DS"] = "20200" + pitcher["GDAY_DS"].astype(str)
for i in range(len(team_list)) :
    pitcher.loc[team_list["T_NM"][i] == pitcher["T_ID"], ["T_ID"]] = team_list["T_ID"][i]
pitcher["WLS"] = pitcher["RES"].str.split(' ').str[0]
pitcher["INN2"] = (pitcher["INN2"].astype(str).str.split('.').str[0].astype(int) * 3) + pitcher["INN2"].astype(str).str.split('.').str[1].astype(int)

In [8]:
pitcher["TB_SC"] = pitcher["VS_T_ID"].str.find('@')
pitcher.loc[pitcher["TB_SC"] == 0, "TB_SC"] = "T"
pitcher.loc[pitcher["TB_SC"] == -1, "TB_SC"] = "B"
player_pitcher = pitcher

batter["TB_SC"] = batter["VS_T_ID"].str.find('@')
batter.loc[batter["TB_SC"] == 0, "TB_SC"] = "T"
batter.loc[batter["TB_SC"] == -1, "TB_SC"] = "B"
player_batter = batter

In [9]:
player_pitcher["wrong"] = 0
player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) >= 20200812) & (player_pitcher["NAME"] == "문경찬") & (player_pitcher["T_ID"] == "HT"), "wrong"] = 1
player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) < 20200812) & (player_pitcher["NAME"] == "문경찬") & (player_pitcher["T_ID"] == "NC"), "wrong"] = 1

player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) >= 20200812) & (player_pitcher["NAME"] == "박정수") & (player_pitcher["T_ID"] == "HT"), "wrong"] = 1
player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) < 20200812) & (player_pitcher["NAME"] == "박정수") & (player_pitcher["T_ID"] == "NC"), "wrong"] = 1

player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) >= 20200812) & (player_pitcher["NAME"] == "장현식") & (player_pitcher["T_ID"] == "NC"), "wrong"] = 1
player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) < 20200812) & (player_pitcher["NAME"] == "장현식") & (player_pitcher["T_ID"] == "HT"), "wrong"] = 1

player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) >= 20200812) & (player_pitcher["NAME"] == "김태진") & (player_pitcher["T_ID"] == "NC"), "wrong"] = 1
player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) >= 20200812) & (player_pitcher["NAME"] == "김태진") & (player_pitcher["T_ID"] == "HT"), "wrong"] = 1

player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) >= 20200813) & (player_pitcher["NAME"] == "오태곤") & (player_pitcher["T_ID"] == "KT"), "wrong"] = 1
player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) < 20200813) & (player_pitcher["NAME"] == "오태곤") & (player_pitcher["T_ID"] == "SK"), "wrong"] = 1

player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) >= 20200813) & (player_pitcher["NAME"] == "이홍구") & (player_pitcher["T_ID"] == "SK"), "wrong"] = 1
player_pitcher.loc[(player_pitcher["GDAY_DS"].astype(int) < 20200813) & (player_pitcher["NAME"] == "이홍구") & (player_pitcher["T_ID"] == "KT"), "wrong"] = 1

player_pitcher.loc[(player_pitcher["NAME"] == "홍건희") & (player_pitcher["T_ID"] == "HT")] = 1
player_pitcher.loc[(player_pitcher["NAME"] == "이태양") & (player_pitcher["T_ID"] == "HH")] = 1

player_pitcher = player_pitcher[player_pitcher["wrong"] == 0].drop("wrong", axis = "columns")
player_pitcher = player_pitcher.drop_duplicates()

In [10]:
player_batter["wrong"] = 0
player_batter.loc[(player_batter["GDAY_DS"].astype(int) >= 20200813) & (player_batter["NAME"] == "오태곤") & (player_batter["T_ID"] == "KT"), "wrong"] = 1
player_batter.loc[(player_batter["GDAY_DS"].astype(int) < 20200813) & (player_batter["NAME"] == "오태곤") & (player_batter["T_ID"] == "SK"), "wrong"] = 1

player_batter.loc[(player_batter["GDAY_DS"].astype(int) >= 20200812) & (player_batter["NAME"] == "김태진") & (player_batter["T_ID"] == "NC"), "wrong"] = 1
player_batter.loc[(player_batter["GDAY_DS"].astype(int) < 20200812) & (player_batter["NAME"] == "김태진") & (player_batter["T_ID"] == "HT"), "wrong"] = 1

player_batter.loc[(player_batter["GDAY_DS"].astype(int) >= 20200813) & (player_batter["NAME"] == "이홍구") & (player_batter["T_ID"] == "SK"), "wrong"] = 1
player_batter.loc[(player_batter["GDAY_DS"].astype(int) < 20200813) & (player_batter["NAME"] == "이홍구") & (player_batter["T_ID"] == "KT"), "wrong"] = 1

player_batter.loc[(player_batter["NAME"] == "이흥련") & (player_batter["T_ID"] == "OB"), "wrong"] = 1
player_batter.loc[(player_batter["NAME"] == "노수광") & (player_batter["T_ID"] == "SK"), "wrong"] = 1
player_batter = player_batter[player_batter["wrong"] == 0].drop("wrong", axis = "columns")
player_batter = player_batter.drop_duplicates()

In [11]:
player_batter["HEADER_NO"] = player_batter.groupby(["GDAY_DS", "T_ID", "NAME"]).cumcount()
player_pitcher["HEADER_NO"] = player_pitcher.groupby(["GDAY_DS", "T_ID", "NAME"]).cumcount()
player_batter["VS_T_ID"] = player_batter["VS_T_ID"].replace('\@', '', regex = True)

In [12]:
batter_pa["타자"] = batter_pa["타자"].str.split(' ').str[1]
batter_pa["날짜"] = batter_pa["날짜"].replace('\-', '', regex = True)
batter_pa.columns = ["GDAY_DS", "VS_T_ID", "INN", "PIT", "NAME"]
batter_pa = batter_pa[batter_pa["GDAY_DS"].astype(int) > 20200719].reset_index(drop = True)
batter_pa = pd.merge(batter_pa, player_batter[["GDAY_DS", "VS_T_ID", "NAME", "HEADER_NO"]], how = "left")
PA = batter_pa.groupby(["GDAY_DS", "VS_T_ID", "NAME", "HEADER_NO"]).count().reset_index()
PA = PA.drop("INN", axis = "columns")
PA.rename(columns = {"PIT" : "PA"}, inplace = True)

In [13]:
player_batter = pd.merge(player_batter, PA, how = "left")
player_batter["PA"] = player_batter["PA"].fillna(0)
player_batter["AGE_VA"] = 2020 - player_batter["AGE"].str.extract('(^\d*)').values.astype(int)
player_batter = player_batter.drop(["VS_T_ID", "RES", "POSITION", "AGE"], axis = "columns")
player_batter[["START_CK", "AB", "RUN", "HIT", "H2", "H3", "HR", "TB", "RBI", "SB", "CS", "BB", "HP", "IB", "KK", "GD", "SH", "SF", "AGE_VA", "MONEY", "PA"]] = \
player_batter[["START_CK", "AB", "RUN", "HIT", "H2", "H3", "HR", "TB", "RBI", "SB", "CS", "BB", "HP", "IB", "KK", "GD", "SH", "SF", "AGE_VA", "MONEY", "PA"]].astype(float)

In [14]:
team_batter = player_batter.groupby(["GDAY_DS", "T_ID", "HEADER_NO"]).sum().reset_index()
team_batter = team_batter.drop(["MONEY", "AGE_VA"], axis = "columns")
team_batter = pd.merge(team_batter, player_batter[["GDAY_DS", "T_ID", "HEADER_NO", "TB_SC"]].drop_duplicates())

In [15]:
player_pitcher["AGE_VA"] = 2020 - player_pitcher["AGE"].str.extract('(^\d*)').values.astype(int)
player_pitcher = player_pitcher.drop(["VS_T_ID", "RES", "AGE"], axis = "columns")
player_pitcher[["INN2", "R", "ER", "PA", "AB", "HIT", "H2", "H3", "HR", "BB", "IB", "HP", "KK", "BF", "WHIP", "MONEY"]] =\
player_pitcher[["INN2", "R", "ER", "PA", "AB", "HIT", "H2", "H3", "HR", "BB", "IB", "HP", "KK", "BF", "WHIP", "MONEY"]].astype(float)

In [16]:
team_pitcher = player_pitcher.groupby(["GDAY_DS", "T_ID", "HEADER_NO"]).sum().reset_index()
team_pitcher = team_pitcher.drop(["MONEY", "AGE_VA"], axis = "columns")
team_pitcher = pd.merge(team_pitcher, player_pitcher[["GDAY_DS", "T_ID", "HEADER_NO", "TB_SC", "WLS"]].drop_duplicates())
csum_player_pitcher = player_pitcher.groupby(["T_ID", "GDAY_DS", "HEADER_NO", "NAME"]).sum().groupby(["T_ID", "NAME"]).cumsum().reset_index()

In [66]:
player_batter.to_csv("./crawled_player_batter.csv", index = False, encoding = "utf-8-sig")
player_pitcher.to_csv("./crawled_player_pitcher.csv", index = False, encoding = "utf-8-sig")
team_batter.to_csv("./crawled_team_batter.csv", index = False, encoding = "utf-8-sig")
team_pitcher.to_csv("./crawled_team_pitcher.csv", index = False, encoding = "utf-8-sig")
csum_player_pitcher.to_csv("./crawled_csum_player_pitcher.csv", index = False, encoding = "utf-8-sig")